In [1]:
## NEED TO RUN THIS TO GET CHOLMOD TO WORK
import ctypes

CUDA_LIB = "/share/software/user/open/cuda/12.2.0/targets/x86_64-linux/lib"
OPENBLAS = "/share/software/user/open/openblas/0.3.20/lib"
SUITESPARSE = "/share/software/user/open/suitesparse/7.4.0/lib64"

libs_to_load = [
    f"{OPENBLAS}/libopenblas.so.0",
    f"{CUDA_LIB}/libnvrtc.so.12",
    f"{CUDA_LIB}/libcublas.so.12",
    f"{CUDA_LIB}/libcublasLt.so.12",
    f"{CUDA_LIB}/libnvrtc-builtins.so.12.2",
    f"{SUITESPARSE}/libcholmod.so.5",
]

# Load each library with global visibility
for lib in libs_to_load:
    print(f"Loading {lib} ...")
    ctypes.CDLL(lib, mode=ctypes.RTLD_GLOBAL)

# Import sksparse safely
import sksparse.cholmod as cholmod
print("✓ sksparse imported successfully")


Loading /share/software/user/open/openblas/0.3.20/lib/libopenblas.so.0 ...
Loading /share/software/user/open/cuda/12.2.0/targets/x86_64-linux/lib/libnvrtc.so.12 ...
Loading /share/software/user/open/cuda/12.2.0/targets/x86_64-linux/lib/libcublas.so.12 ...
Loading /share/software/user/open/cuda/12.2.0/targets/x86_64-linux/lib/libcublasLt.so.12 ...
Loading /share/software/user/open/cuda/12.2.0/targets/x86_64-linux/lib/libnvrtc-builtins.so.12.2 ...
Loading /share/software/user/open/suitesparse/7.4.0/lib64/libcholmod.so.5 ...
✓ sksparse imported successfully


In [2]:
import cvxpy as cp
import numpy as np
import time
import clarabel
import torch
import pypsa
import pycupdlp.pycupdlp as pycupdlp
import matplotlib.pyplot as plt
import jax.numpy as jnp
from mpax import create_lp, create_qp, raPDHG, r2HPDHG
from zap.admm import ADMMSolver, WeightedADMMSolver
from zap.conic.cone_bridge import ConeBridge
import scipy.sparse as sp
from experiments.conic_solve.benchmarks.max_flow_benchmark import MaxFlowBenchmarkSet
from zap.importers.pypsa import load_pypsa_network, parse_buses


from zap.conic.cone_utils import get_standard_conic_problem, stack_cvxpy_colwise



/home/groups/ramr/akshay81/poetry_cache/virtualenvs/zap-eFcx8M23-py3.12/lib/python3.12/site-packages/diffcp/cone_program.py:207: SyntaxWarning: invalid escape sequence '\i'
  """Solves a cone program, returns its derivative as an abstract linear map.
/home/groups/ramr/akshay81/poetry_cache/virtualenvs/zap-eFcx8M23-py3.12/lib/python3.12/site-packages/pypsa/linopf.py:1253: SyntaxWarning: invalid escape sequence '\ '
  n.objective_f.write("\ LOPF \n\nmin\nobj:\n")


In [3]:
import jax, jaxlib, sys
print(sys.executable)
print(jaxlib.__version__)
print(jax.default_backend(), jax.devices())


/home/groups/ramr/akshay81/poetry_cache/virtualenvs/zap-eFcx8M23-py3.12/bin/python
0.4.36
gpu [CudaDevice(id=0)]


In [4]:
def osqp_to_cupdlp(problem, lift_singletons=True, use_mpax=False):
    """
    Convert CVXPY/OSQP canonical data to the argument list required by
    pycupdlp.cupdlp.loadData.

    Parameters
    ----------
    probdata : dict
        Output of `problem.get_problem_data(solver=cp.OSQP)`.
    lift_singletons : bool, default True
        If True, rows of F with a single ±1 are turned into explicit
        variable bounds and removed from the matrix.

    Returns
    -------
    A   : scipy.sparse.csc_matrix
    c   : np.ndarray
    rhs : np.ndarray
    lb  : np.ndarray
    ub  : np.ndarray
    neqs: int
    """
    # Parse problem
    probdata, chain, inv_data = problem.get_problem_data(solver=cp.OSQP)
    offset = inv_data[3]["offset"]


    # Shorthands
    q   = probdata["q"].astype(float)
    Aeq = probdata["A"].tocsc()
    beq = probdata["b"].astype(float)
    F   = probdata["F"].tocsr()
    G   = probdata["G"].astype(float)

    n_var = probdata["n_var"]
    n_eq  = Aeq.shape[0]

    # ------------------------------------------------------------------
    # Variable bounds from CVXPY itself
    # ------------------------------------------------------------------
    lb = np.full(n_var, -np.inf)
    ub = np.full(n_var,  np.inf)
    if probdata["lower_bounds"] is not None:
        lb = np.maximum(lb, probdata["lower_bounds"])
    if probdata["upper_bounds"] is not None:
        ub = np.minimum(ub, probdata["upper_bounds"])

    # ------------------------------------------------------------------
    # Optionally lift singleton rows  (±1·x_j ≤ g_i)
    # ------------------------------------------------------------------
    keep_row = np.ones(F.shape[0], dtype=bool)
    if lift_singletons:
        row_nnz = np.diff(F.indptr)
        single  = np.flatnonzero(row_nnz == 1)

        pos     = F.indptr[single]
        cols    = F.indices[pos]
        vals    = F.data[pos]

        plus_mask  = vals == +1
        minus_mask = vals == -1

        ub[cols[plus_mask]]  = np.minimum(ub[cols[plus_mask]],  G[single[plus_mask]])
        lb[cols[minus_mask]] = np.maximum(lb[cols[minus_mask]], -G[single[minus_mask]])

        keep_row[single[plus_mask | minus_mask]] = False

    # ------------------------------------------------------------------
    # Inequality block:  –F x ≥ –G
    # ------------------------------------------------------------------
    F_kept = (-F[keep_row]).tocsc()
    G_kept = (-G[keep_row])

    # ------------------------------------------------------------------
    # Stack [inequalities; equalities]  →  A , rhs
    # ------------------------------------------------------------------
    A   = sp.vstack([Aeq, F_kept], format="csc")
    rhs = np.concatenate([beq, G_kept])
    neqs = n_eq                    # equalities are the bottom block

    # CuPDLP minimises, same orientation as OSQP
    cost = q

    if use_mpax:
        return Aeq, cost, beq, G_kept, F_kept, lb, ub, offset # A, c, b, h, G, l, u
    else:
        return A, cost, rhs, lb, ub, neqs, offset


In [5]:
PYPSA_NETW0RK_PATH = '/scratch/groups/ramr/akshay81/data/pypsa-networks/western_small/network_2021.nc'
pn = pypsa.Network(PYPSA_NETW0RK_PATH)
snapshots = pn.generators_t.p_max_pu.index
time_horizon = 720
snapshots_test = snapshots[0:time_horizon]
pypsa_kwargs = {}
pypsa_net_test, pypsa_devices_test = load_pypsa_network(pn, snapshots_test,  power_unit=1.0e3, cost_unit=100.0, **pypsa_kwargs)

INFO:pypsa.io:Imported network network_2021.nc has buses, carriers, generators, lines, links, loads, storage_units


Dropping empty generators.


In [16]:
pypsa_problem = pypsa_net_test.dispatch(pypsa_devices_test, time_horizon=time_horizon, solver=cp.CLARABEL, add_ground=False, return_problem_only=True)
A_pypsa, c_pypsa, rhs_pypsa, lb_pypsa, ub_pypsa, neqs_pypsa, offset_pypsa = osqp_to_cupdlp(pypsa_problem)
A_mpax, c_mpax, b_mpax, h_mpax, G_mpax, lb_mpax, ub_mpax, offset_mpax = osqp_to_cupdlp(pypsa_problem, use_mpax=True)




In [17]:
## MPAX test without batching on PyPSA problem
mpax_nobatch_lp = create_lp(c_mpax, A_mpax, b_mpax, G_mpax, h_mpax, lb_mpax, ub_mpax)
mpax_solver = r2HPDHG(eps_abs=1e-4, eps_rel=1e-4, verbose=True)
result_mpax = mpax_solver.optimize(mpax_nobatch_lp)



Problem after rescaling (Ruiz iterations = 10, l2_norm_rescaling = False):
There are 5820518 variables, 4873036 constraints (including 3409996 equalities) and 13239436 nonzero coefficients.
Absolute value of nonzero constraint matrix elements:
  largest=0.768862, smallest=0.000000, avg=0.354463
Constraint matrix, infinity norm:
  max_col=0.768862, min_col=0.223607, max_row=0.768862, min_row=0.223607
Absolute value of objective matrix elements:  largest=0.000000, smallest=0.000000, avg=nan
Absolute value of objective vector elements:
  largest=10.000000, smallest=0.000000, avg=0.534669
Absolute value of rhs vector elements:
  largest=1.219200, smallest=0.000000, avg=0.000002
Gap between upper and lower bounds:
  # finite= 2511398 of 5820518, largest= nan, smallest= nan, avg= 5.202895
Preconditioning Time (seconds): 1.91e+01
runtime | residuals                 | solution information      | relative residuals      |
#iter   | pr_norm  du_norm  gap     | pr_obj   pr_norm  du_norm | rel_pr 

In [10]:
result_mpax.primal_solution.shape

(1455158,)

In [6]:
## Test MPAX with batching—created the batched data
snapshots = pn.generators_t.p_max_pu.index
start_t = 0
A_list = []
b_list = []
c_list = []
h_list = []
G_list = []
lb_list = []
ub_list = []
offset_list = []
solution_histories = []
time_horizon = 720
for i in range(720):
    end_t = i+1
    snapshots_test = snapshots[start_t:end_t]
    pypsa_kwargs = {}
    pypsa_net_test, pypsa_devices_test = load_pypsa_network(pn, snapshots_test,  power_unit=1.0e3, cost_unit=100.0, **pypsa_kwargs)
    start_t = end_t
    pypsa_problem = pypsa_net_test.dispatch(pypsa_devices_test, time_horizon=1, solver=cp.CLARABEL, add_ground=False, return_problem_only=True)
    A_mpax, c_mpax, b_mpax, h_mpax, G_mpax, lb_mpax, ub_mpax, offset_mpax = osqp_to_cupdlp(pypsa_problem, use_mpax=True)

    A_list.append(A_mpax)
    b_list.append(b_mpax)
    c_list.append(c_mpax)
    h_list.append(h_mpax)
    G_list.append(G_mpax)
    lb_list.append(lb_mpax)
    ub_list.append(ub_mpax)
    offset_list.append(offset_mpax)

Dropping empty generators.
Dropping empty generators.
Dropping empty generators.
Dropping empty generators.
Dropping empty generators.
Dropping empty generators.
Dropping empty generators.
Dropping empty generators.
Dropping empty generators.
Dropping empty generators.
Dropping empty generators.
Dropping empty generators.
Dropping empty generators.
Dropping empty generators.
Dropping empty generators.
Dropping empty generators.
Dropping empty generators.
Dropping empty generators.
Dropping empty generators.
Dropping empty generators.
Dropping empty generators.
Dropping empty generators.
Dropping empty generators.
Dropping empty generators.
Dropping empty generators.
Dropping empty generators.
Dropping empty generators.
Dropping empty generators.
Dropping empty generators.
Dropping empty generators.
Dropping empty generators.
Dropping empty generators.
Dropping empty generators.
Dropping empty generators.
Dropping empty generators.
Dropping empty generators.
Dropping empty generators.
D

In [7]:
def all_equal_dense(arr_list, rtol=1e-9, atol=1e-12):
    ref = arr_list[0]
    return all(np.allclose(x, ref, rtol=rtol, atol=atol) for x in arr_list[1:])

def all_equal_sparse(mat_list, rtol=1e-9, atol=1e-12):
    ref = mat_list[0].tocoo()
    for M in mat_list[1:]:
        M = M.tocoo()
        if not (np.array_equal(M.row, ref.row) and np.array_equal(M.col, ref.col)):
            return False
        if not np.allclose(M.data, ref.data, rtol=rtol, atol=atol):
            print("sparsity pattern is the same")
            return False        # pattern same but data differ
    return True


In [8]:
print("A identical :", all_equal_sparse(A_list))
print("G identical :", all_equal_sparse(G_list))
print("b identical :", all_equal_dense(b_list))
print("c identical :", all_equal_dense(c_list))
print("h identical :", all_equal_dense(h_list))
print("lb identical:", all_equal_dense(lb_list))
print("ub identical:", all_equal_dense(ub_list))
print("offset identical:", all_equal_dense(offset_list))


A identical : True
G identical : True
b identical : True
c identical : True
h identical : True
lb identical: False
ub identical: False
offset identical: False


In [9]:
len(A_list)
print(A_list[0].shape)

(1260, 2059)


In [9]:
## Test MPAX with batching—solve the batched problem

A = A_list[0]
G = G_list[0]
b = jnp.array(b_list[0])
c = jnp.array(c_list[0])
h = jnp.array(h_list[0])
lb_batch = jnp.stack(lb_list)
ub_batch = jnp.stack(ub_list)
B, n = lb_batch.shape
solver = r2HPDHG(eps_abs=1e-4, eps_rel=1e-4, verbose=False)


def solve_one(lb_i, ub_i):
    """
    Only the LB and UB changes between problem instances
    """
    lp = create_lp(c,A, b, G, h, lb_i, ub_i)
    res = solver.optimize(lp)
    x_star = res.primal_solution
    obj = jnp.dot(c, x_star)
    return x_star, obj


batched_solve = jax.vmap(solve_one, in_axes=(0, 0), out_axes=(0, 0))
batched_solve = jax.vmap(solve_one)


primal_batch, obj_batch = batched_solve(lb_batch, ub_batch)

print("all solutions:", primal_batch.shape) # (B, n)
print("all objectives:", obj_batch.shape)     # (B,)


all solutions: (720, 2059)
all objectives: (720,)


In [21]:
ub_batch.shape

(720, 2059)

In [11]:
## Do max flow stuff

## Create a large problem that is valid using max flow benchmark set 
n = 10000
base_seed = 42
max_flow_benchmark = MaxFlowBenchmarkSet(num_problems=1, n=n, base_seed=base_seed)

for prob in max_flow_benchmark:
    problem = prob
num_variables = sum(np.prod(var.shape) for var in problem.variables())
num_constraints = sum(constraint.size for constraint in problem.constraints)
nnz = num_variables # This is the actual number of edges
total_possible_edges = float(n*(n - 1))
density = nnz/total_possible_edges
sparsity = 1 - density
print(f'Generated a valid network with {n} nodes using starting seed {base_seed}')
print(f"Actual Number of Edges: {nnz}")
print(f"Total Possible Edges: {total_possible_edges}")
print(f"Graph sparsity: {sparsity}")
print(f"Graph density: {density}")
print(f"Number of Variables: {num_variables}")
print(f"Number of Constraints: {num_constraints}")

Generated a valid network with 10000 nodes using starting seed 42
Actual Number of Edges: 183993
Total Possible Edges: 99990000.0
Graph sparsity: 0.9981598859885988
Graph density: 0.0018401140114011401
Number of Variables: 183993
Number of Constraints: 193993


In [7]:
A_mflow, c_mflow, b_mflow, h_mflow, G_mflow, lb_mflow, ub_mflow, offset_mflow = osqp_to_cupdlp(problem, use_mpax=True)


In [8]:
## MPAX test on a large net flow problem
mpax_nobatch_maxflow = create_lp(c_mflow, A_mflow, b_mflow, G_mflow, h_mflow, lb_mflow, ub_mflow)
mpax_solver = r2HPDHG(eps_abs=1e-4, eps_rel=1e-4, verbose=True)
result_mpax_mflow = mpax_solver.optimize(mpax_nobatch_maxflow)

Problem after rescaling (Ruiz iterations = 10, l2_norm_rescaling = False):
There are 27630033 variables, 1000000 constraints (including 1000000 equalities) and 55260066 nonzero coefficients.
Absolute value of nonzero constraint matrix elements:
  largest=0.144338, smallest=0.000000, avg=0.094905
Constraint matrix, infinity norm:
  max_col=0.144338, min_col=0.076249, max_row=0.144338, min_row=0.072932
Absolute value of objective matrix elements:  largest=0.000000, smallest=0.000000, avg=nan
Absolute value of objective vector elements:
  largest=0.707107, smallest=0.000000, avg=0.000000
Absolute value of rhs vector elements:
  largest=0.000000, smallest=0.000000, avg=0.000000
Gap between upper and lower bounds:
  # finite= 0 of 27630033, largest= nan, smallest= nan, avg= nan
Preconditioning Time (seconds): 1.17e+01
runtime | residuals                 | solution information      | relative residuals      |
#iter   | pr_norm  du_norm  gap     | pr_obj   pr_norm  du_norm | rel_pr  rel_du  r

In [12]:
cone_params_batched = stack_cvxpy_colwise([problem]*20)

In [13]:
# Solve the batched max flow instance using zap
machine = 'cuda'
dtype = torch.float32
cone_bridge_batched = ConeBridge(cone_params_batched, ruiz_iters = 0)
cone_admm_devices_batched = [d.torchify(machine=machine, dtype=dtype) for d in cone_bridge_batched.devices]
cone_admm = ADMMSolver(
    machine=machine,
    dtype=dtype,
    atol=1e-6,
    rtol=1e-6,
    num_iterations=10000,
)
start_time = time.time()
cone_solution_admm_batched, cone_history_admm_batched = cone_admm.solve(
    net=cone_bridge_batched.net, devices=cone_admm_devices_batched, time_horizon=cone_bridge_batched.time_horizon
)
end_time = time.time()
solve_time = end_time - start_time
obj_val = cone_solution_admm_batched.objective
print(f"Objective value: {obj_val}")
print(f"Time taken: {solve_time:.4f} seconds")


Did not converge. Ran for 10000 iterations.
Objective value: -3172.8857421875
Time taken: 42.6536 seconds


In [16]:
# Solve the batched max flow instance using mpax

A_mflow, c_mflow, b_mflow, h_mflow, G_mflow, lb_mflow, ub_mflow, offset_mflow = osqp_to_cupdlp(problem, use_mpax=True)
B = 20

A = [A_mflow] * B
G = [G_mflow] * B
b = jnp.tile(jnp.array(b_mflow), (B,1))
c = jnp.tile(jnp.array(c_mflow), (B,1))
h = jnp.tile(jnp.array(h_mflow), (B,1))
lb_batch = jnp.tile(jnp.array(lb_mflow), (B,1))
ub_batch = jnp.tile(jnp.array(ub_mflow), (B,1))
solver = r2HPDHG(eps_abs=1e-4, eps_rel=1e-4, verbose=False)


def solve_one(b_i, c_i, h_i, lb_i, ub_i):
    """
    Only the LB and UB changes between problem instances
    """
    lp = create_lp(c_i, A_mflow, b_i, G_mflow, h_i, lb_i, ub_i)
    res = solver.optimize(lp)
    x_star = res.primal_solution
    obj = jnp.dot(c_i, x_star)
    return x_star, obj


batched_solve = jax.vmap(solve_one, in_axes=(0, 0, 0, 0, 0), out_axes=(0, 0))

primal_batch, obj_batch = batched_solve(b, c, h, lb_batch, ub_batch)

print("all solutions:", primal_batch.shape) # (B, n)
print("all objectives:", obj_batch.shape)     # (B,)


all solutions: (20, 183993)
all objectives: (20,)


In [13]:
G_mflow

<0x183993 sparse matrix of type '<class 'numpy.float64'>'
	with 0 stored elements in Compressed Sparse Column format>